<a href="https://colab.research.google.com/github/jlgelpi/BioPhysics/blob/master/6m0j_check_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structure checking tutorial (6m0j)

A complete checking analysis of a single structure follows.
use .revert_changes() at any time to reload the original structure

Structure checking is a key step before setting up a protein system for simulations.
A number of normal issues found in structures at Protein Data Bank may compromise the success of the simulation, or may suggest that longer equilibration procedures are necessary.

The biobb_structure_checking modules allow to
- Do basic manipulations on structures (selection of models, chains, alternative locations
- Detect and fix amide assignments, wrong chirality
- Detect and fix protein backbone issues (missing fragments, and atoms, capping)
- Detect and fix missing side-chain atoms
- Add hydrogen atoms according to several criteria
- Detect and classify clashes
- Detect possible SS bonds

biobb_structure_checking modules can used at the command line biobb_structure_checking/bin/check_structure


## Installation

In [ ]:
!pip install biopython psutils biobb_structure_checking nglview

#### Basic imports and initialization

In [ ]:
import biobb_structure_checking
from biobb_structure_checking.constants import help, set_defaults
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path = biobb_structure_checking.__path__[0]
args = set_defaults(base_dir_path)

## General help

In [ ]:
help()


BioBB's check_structure.py performs MDWeb structure checking set as a command line
utility.

commands:     Help on available commands
command_list: Run all tests from conf file or command line list
checkall:     Perform all checks without fixes
load:         Stores structure on local cache and provides basic statistics

1. System Configuration
sequences [--output_fasta FASTA]
    Print canonical and structure sequences in FASTA format
    --output_fasta Write sequences found in an externla FASTA file
models [--select model_num] [--superimpose] [--save_split]
    Detect/Select Models
    --superimpose Superimposes currently selected models
    --save_split Split models as separated output files.
    --build_complex Build an actual complex from Biounits stored as collection of models
chains [--select chain_ids | molecule_type] [--rename (auto|new label)]
    [--renumber (auto | [A:]ini0[-fin0]=[B:]ini1)] [--rem_inscodes]
    Detect/Select Chains
    --select chains ids or chain types (P

Set input (PDB or local file, pdb or mmCif formats allowed) and output (local file, pdb format).  
Use pdb:pdbid for downloading structure from PDB (RCSB)

In [ ]:
base_path = './'
args['input_structure_path'] = 'pdb:6m0j'
args['output_structure_path'] = base_path + '6m0j_fixed.pdb'
args['output_structure_path_charges'] = base_path + '6m0j_fixed.pdbqt'

Initializing checking engine, loading structure and showing statistics

In [ ]:
st_c = StructureChecking(base_dir_path,args)

Structure exists: 'tmpPDB/m0/6m0j.cif' 
Structure pdb:6m0j loaded
 PDB id: 6M0J 
 Title: Crystal structure of SARS-CoV-2 spike receptor-binding domain bound with ACE2
 Experimental method: X-RAY DIFFRACTION
 Keywords: VIRAL PROTEIN/HYDROLASE
 Resolution (A): 2.4500

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  878
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  87
 Num. ligands or modified residues:  7
 Num. water mol.:  80
 Num. atoms:  6558
Metal/Ion residues found
 ZN A901
 CL A902
Small mol ligands found
NAG A903
NAG A904
NAG A905
NAG A906
NAG E601


#### models
Checks for the presence of models in the structure.
MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.  
Use models('--select N') to select model num N for further analysis

In [ ]:
st_c.models()

Running models.
Detected 1 Model(s)
Found Single model


#### chains
Checks for chains (also obtained from print_stats), and allow to select one or more.   
MD simulations are usually performed with complete structures. However input structure may contain several copies of the system, or contains additional chains like peptides or nucleic acids that may be removed.
Use chains('X,Y') to select chain(s) X and Y to proceed

In [ ]:
st_c.chains()

Running chains.
Detected 2 Chain(s)
 A: Protein
 E: Protein


#### altloc
Checks for the presence of residues with alternative locations. Atoms with alternative coordinates and their occupancy are reported.  
MD simulations requires a single position for each atom.  
Use altloc('occupancy | alt_ids | list of res:id) to select the alternative


In [ ]:
st_c.altloc()

Running altloc.
Detected 2 residues with alternative location labels
HIS A228
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  ND1  A (0.50) B (0.50)
  CD2  A (0.50) B (0.50)
  CE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
GLN E493
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  CD   A (0.50) B (0.50)
  OE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)


We need to choose one of the alternative forms for each residue

In [ ]:
st_c.altloc('occupancy')

Running altloc. Options: occupancy
Detected 2 residues with alternative location labels
HIS A228
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  ND1  A (0.50) B (0.50)
  CD2  A (0.50) B (0.50)
  CE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
GLN E493
  CA   A (0.50) B (0.50)
  CB   A (0.50) B (0.50)
  CG   A (0.50) B (0.50)
  CD   A (0.50) B (0.50)
  OE1  A (0.50) B (0.50)
  NE2  A (0.50) B (0.50)
Selecting location occupancy


In [ ]:
st_c.altloc()

Running altloc.
Detected no residues with alternative location labels


#### metals
Detects HETATM being metal ions allow to selectively remove them.  
To remove use metals (' All | None | metal_type list | residue list ')

In [ ]:
st_c.metals()

Running metals.
Found 1 Metal ions
  ZN A901.ZN 


#### ligands
Detects HETATM (excluding Water molecules) to selectively remove them.  
To remove use ligands('All | None | Residue List (by id, by num)')


In [ ]:
st_c.ligands()

Running ligands.
Detected 7 Ligands
  ZN A901
  CL A902
 NAG A903
 NAG A904
 NAG A905
 NAG A906
 NAG E601


In [ ]:
st_c.ligands('All')

Running ligands. Options: All
Detected 7 Ligands
  ZN A901
  CL A902
 NAG A903
 NAG A904
 NAG A905
 NAG A906
 NAG E601
Ligands removed All (7)


In [ ]:
st_c.ligands()

Running ligands.
No ligands found


#### rem_hydrogen
Detects and remove hydrogen atoms.
MD setup can be done with the original H atoms, however to prevent from non standard labelling, remove them is safer.  
To remove use rem_hydrogen('yes')


In [ ]:
st_c.rem_hydrogen()

Running rem_hydrogen.
No residues with Hydrogen atoms found


#### water
Detects water molecules and allows to remove them
Crystallographic water molecules may be relevant for keeping the structure, however in most cases only some of them are required. These can be later added using other methods (titration) or manually.

To remove water molecules use water('yes')


In [ ]:
st_c.water()

Running water.
Detected 80 Water molecules


In [ ]:
st_c.water("yes")

Running water. Options: yes
Detected 80 Water molecules
Removed 80 Water molecules


#### amide
Amide terminal atoms in Asn ang Gln residues can be labelled incorrectly.  
amide suggests possible fixes by checking the sourrounding environent.

To fix use amide ('All | None | residue_list')

Note that the inversion of amide atoms may trigger additional contacts.

In [ ]:
st_c.amide()

Running amide.
Found 7 unusual contact(s) involving amide atoms
 LYS A31.NZ   GLN E493.NE2    2.926 A
 GLN A42.NE2  GLN E498.NE2    2.927 A
 ASN A103.OD1 ASN A194.OD1    2.807 A
 ASN A134.OD1 GLU A140.OE2    2.785 A
 ASN A134.ND2 ASN A137.N      3.082 A
 GLU A150.O   ASN A154.OD1    2.895 A
 ARG E357.NH1 ASN E394.ND2    2.963 A


Fix all amide residues and recheck

In [ ]:
st_c.amide('auto')

Running amide. Options: auto
Found 7 unusual contact(s) involving amide atoms
 LYS A31.NZ   GLN E493.NE2    2.926 A
 GLN A42.NE2  GLN E498.NE2    2.927 A
 ASN A103.OD1 ASN A194.OD1    2.807 A
 ASN A134.OD1 GLU A140.OE2    2.785 A
 ASN A134.ND2 ASN A137.N      3.082 A
 GLU A150.O   ASN A154.OD1    2.895 A
 ARG E357.NH1 ASN E394.ND2    2.963 A
Fixing automatically
Initial contact score: 1.177
Clustering amide residues
6 cluster(s) found, exploring...
Cluster 1:ASN A134
New score: 0.837, fixed residue(s): ASN A134
Cluster 2:ASN A154
New score: 0.718, fixed residue(s): ASN A154
Cluster 3:ASN A103, ASN A194
New score: 0.464, fixed residue(s): ASN A194
Cluster 4:GLN A42, GLN E498
New score: 0.231, fixed residue(s): GLN E498
Cluster 5:ASN E394
Score not improved, skipping
Cluster 6:GLN E493
New score: 0.114, fixed residue(s): GLN E493
Amide residues fixed auto (5)
Rechecking
Found 1 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    2.963 A


#### chiral
Side chains of Thr and Ile are chiral, incorrect atom labelling lead to the wrong chirality.  
To fix use chiral('All | None | residue_list')

In [ ]:
st_c.chiral()

Running chiral.
Found no residues with incorrect side-chain chirality


#### Backbone
Detects and fixes several problems with the backbone
use any of
--fix_atoms All|None|Residue List
--fix_chain All|None|Break list
--add_caps All|None|Terms|Breaks|Residue list
--no_recheck
--no_check_clashes


In [ ]:
st_c.backbone()

Running backbone.
Found 2 Residues with missing backbone atoms
 ASP A615   OXT
 GLY E526   OXT
No backbone breaks
No unexpected backbone links


In [ ]:
help('backbone')

usage: backbone [-h] [--fix_atoms FIX_ATOMS] [--fix_chain FIX_CHAIN]
                [--add_caps ADD_CAPS] [--no_check_clashes]
                [--extra_gap EXTRA_GAP] [--no_recheck]


In [ ]:
opts = {
    'fix_atoms':'all',
    'fix_chain':'none',
    'add_caps':'none'
}
st_c.backbone(opts)

# Alternatively
# st_c.backbone('--fix_atoms All --fix_chain none --add_caps none')

Running backbone. Options: {'fix_atoms': 'all', 'fix_chain': 'none', 'add_caps': 'none'}
Found 2 Residues with missing backbone atoms
 ASP A615   OXT
 GLY E526   OXT
No backbone breaks
No unexpected backbone links
Capping terminal ends
True terminal residues: A19,A615,E333,E526
No caps added
Fixing missing backbone atoms
Adding missing backbone atoms
ASP A615
  Adding new atom OXT
GLY E526
  Adding new atom OXT
Fixed 2 backbone atom(s)
Checking for steric clashes
No Severe Steric clashes/covalent bonds detected
No Apolar steric clashes detected
No Polar contacts (acceptors) detected
No Polar contacts (donors) detected
No Positively charged contacts detected
No Negatively charged contacts detected


#### fixside
Detects and re-built missing protein side chains.   
To fix use fixside('All | None | residue_list')

In [ ]:
st_c.fixside()

Running fixside.
Found no residues with missing or unknown side chain atoms


#### getss
Detects possible -S-S- bonds based on distance criteria.
Proper simulation requires those bonds to be correctly set. Use All|None|residueList to mark them

In [ ]:
st_c.getss()

Running getss.
Detected 7 Possible SS Bonds
 CYS A133.SG  CYS A141.SG     4.237
 CYS A344.SG  CYS A361.SG     4.159
 CYS A530.SG  CYS A542.SG     4.095
 CYS E336.SG  CYS E361.SG     4.152
 CYS E379.SG  CYS E432.SG     4.177
 CYS E391.SG  CYS E525.SG     4.191
 CYS E480.SG  CYS E488.SG     4.269


In [ ]:
st_c.getss('all')

Running getss. Options: all
Detected 7 Possible SS Bonds
 CYS A133.SG  CYS A141.SG     4.237
 CYS A344.SG  CYS A361.SG     4.159
 CYS A530.SG  CYS A542.SG     4.095
 CYS E336.SG  CYS E361.SG     4.152
 CYS E379.SG  CYS E432.SG     4.177
 CYS E391.SG  CYS E525.SG     4.191
 CYS E480.SG  CYS E488.SG     4.269


#### Add_hydrogens
 Add Hydrogen Atoms. Auto: std changes at pH 7.0. His->Hie. pH: set pH value
    list: Explicit list as [*:]HisXXHid, Interactive[_his]: Prompts for all selectable residues
    Fixes missing side chain atoms unless --no_fix_side is set
    Existing hydrogen atoms are removed before adding new ones unless --keep_h set.

In [ ]:
st_c.add_hydrogen()

Running add_hydrogen.
Found 226 Residues requiring selection on adding H atoms


In [ ]:
st_c.add_hydrogen('auto')

Running add_hydrogen. Options: auto
Found 226 Residues requiring selection on adding H atoms
Running fixside. Options: --fix all
Found no residues with missing or unknown side chain atoms
Selection: auto
Replacing HIS A34 by HIE
Replacing HIS A195 by HIE
Replacing HIS A228 by HIE
Replacing HIS A239 by HIE
Replacing HIS A241 by HIE
Replacing HIS A265 by HIE
Replacing HIS A345 by HIE
Replacing HIS A373 by HIE
Replacing HIS A374 by HIE
Replacing HIS A378 by HIE
Replacing HIS A401 by HIE
Replacing HIS A417 by HIE
Replacing HIS A493 by HIE
Replacing HIS A505 by HIE
Replacing HIS A535 by HIE
Replacing HIS A540 by HIE
Replacing HIS E519 by HIE


#### clashes
Detects steric clashes based on distance criteria.  
Contacts are classified in:
* Severe/Covalent: Too close atoms, usually indicating superimposed structures,badly modelled regionsor unexpected covalent bonds. Clashes should be fixed, covalent bonds should be considered in further manipulations.
* Apolar: Vdw colissions.Usually fixed during the simulation.
* Polar and ionic. Usually indicate wrong side chain conformations. Usually fixed during the simulation


In [ ]:
st_c.clashes()

Running clashes.
No Severe Steric clashes/covalent bonds detected
4 Apolar steric clashes detected
 HIE A34.CD2  TYR E453.OH     2.860 A
 ASN A121.O   THR A125.CG2    2.890 A
 LEU A333.C   MET A360.O      2.881 A
 TYR E380.O   THR E430.C      2.758 A
5 Polar contacts (acceptors) detected
 MET A152.O   GLY A268.O      3.063 A
 LEU A333.O   MET A360.O      2.881 A
 TYR E351.O   ASP E467.O      3.074 A
 TYR E380.O   THR E430.O      2.728 A
 GLY E485.O   CYX E488.O      3.046 A
1 Polar contacts (donors) detected
 ARG E357.NH1 ASN E394.ND2    2.963 A
No Positively charged contacts detected
No Negatively charged contacts detected


Complete check in a single method

In [ ]:
st_c.checkall()

Running models.
Detected 1 Model(s)
Found Single model
Running chains.
Detected 2 Chain(s)
 A: Protein
 E: Protein
Running inscodes.
Found no residues with insertion codes
Running altloc.
Detected no residues with alternative location labels
Running rem_hydrogen.
Detected 791 Residues containing H atoms
Running add_hydrogen.
Found 209 Residues requiring selection on adding H atoms
Running water.
No water molecules found
Running metals.
No metal ions found
Running ligands.
No ligands found
Running getss.
Detected 7 Possible SS Bonds
 CYX A133.SG  CYX A141.SG     4.237
 CYX A344.SG  CYX A361.SG     4.159
 CYX A530.SG  CYX A542.SG     4.095
 CYX E336.SG  CYX E361.SG     4.152
 CYX E379.SG  CYX E432.SG     4.177
 CYX E391.SG  CYX E525.SG     4.191
 CYX E480.SG  CYX E488.SG     4.269
Running amide.
Found 1 unusual contact(s) involving amide atoms
 ARG E357.NH1 ASN E394.ND2    2.963 A
Running chiral.
Found no residues with incorrect side-chain chirality
Running chiral_bck.
Found no residues 

In [ ]:
st_c.args['output_format'] = 'pdb' # Add the missing output_format to args
st_c.save_structure(args['output_structure_path'])

'./6m0j_fixed.pdb'

In [ ]:
import nglview as nv
from google.colab import output
output.enable_custom_widget_manager()
nv.show_biopython(st_c.strucm.st[0])

NGLWidget()

In [ ]:
st_c.rem_hydrogen('yes')

Running rem_hydrogen. Options: yes
Detected 791 Residues containing H atoms
Hydrogen atoms removed from 791 residues


In [ ]:
opts = {
    'add_mode': 'auto',
    'add_charges': 'CMIP'
}

st_c.add_hydrogen(opts)

#st_c.add_hydrogen('--add_charges ADT --add_mode auto')

Running add_hydrogen. Options: {'add_mode': 'auto', 'add_charges': 'CMIP'}
Found 240 Residues requiring selection on adding H atoms
Running fixside. Options: --fix all
Found no residues with missing or unknown side chain atoms
Selection: auto
Replacing HIS A34 by HIE
Replacing HIS A195 by HIE
Replacing HIS A228 by HIE
Replacing HIS A239 by HIE
Replacing HIS A241 by HIE
Replacing HIS A265 by HIE
Replacing HIS A345 by HIE
Replacing HIS A373 by HIE
Replacing HIS A374 by HIE
Replacing HIS A378 by HIE
Replacing HIS A401 by HIE
Replacing HIS A417 by HIE
Replacing HIS A493 by HIE
Replacing HIS A505 by HIE
Replacing HIS A535 by HIE
Replacing HIS A540 by HIE
Replacing HIS E519 by HIE
Updating partial charges and atom types
Total assigned charge:     -26.00


In [ ]:
args['output_format'] = 'pdbqt'
st_c._save_structure(args['output_structure_path_charges'])

'./6m0j_fixed.pdbqt'

In [ ]:
st_c.print_stats()

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 791 (total 6116 H atoms)
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  12524
